<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/Integrations/Langchain/Chains/Prompt-templates_and_chains.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# Prompt templates and chains

## Introduction

- Notebook to showcase different frameworks of langchain with clarifai models and and SDK functions.

Install clarifai and langchain packages.

In [ ]:
!pip install langchain
!pip install clarifai

You can use several language models from [clarifai](https://clarifai.com/explore/models?filterData=%5B%7B%22field%22%3A%22use_cases%22%2C%22value%22%3A%5B%22llm%22%5D%7D%5D&page=1&perPage=24) platform. Sign up and get your [PAT](https://clarifai.com/settings/security) to access it.

In [ ]:
# use model URL
MODEL_URL="https://clarifai.com/mistralai/completion/models/mistral-7B-OpenOrca"

                     #or

# Use model parameters, change user_id, app_id and model_id to use different models from the clarifai platform.
USER_ID = "mistralai"
APP_ID = "completion"
MODEL_ID = "mistral-7B-OpenOrca"

For the example we are using ***mistral-7B-OpenOrca*** model.

## **mistral-7B-OpenOrca**
This [mistral-7B-OpenOrca](https://clarifai.com/mistralai/completion/models/mistral-7B-OpenOrca) release is a finetune of Mistral-7B base model. It claims to achieve 98% of the eval performance of Llama2-70B.



In [ ]:
import os
os.environ["CLARIFAI_PAT"]="YOUR_CLARIFAI_PAT"

set_debug(True)

In [ ]:
from langchain.globals import set_debug
set_debug(True)

Setting the global debug flag will cause all LangChain components with callback support (chains, models, agents, tools, retrievers) to print the inputs they receive and outputs they generate.

Initialize Clarifai llm class from langchain

In [ ]:
from langchain.llms import Clarifai
llm=Clarifai(user_id=USER_ID, app_id=APP_ID, model_id=MODEL_ID)

##**Prompt template examples**

Prompt template gives basic instructions to the model along with the prompt to language model. It is vital to have proper prompt template in the pipeline of AI apps to obtain desirable responses in certain format which is ready to be used in the workflow.

In [ ]:
#basic structure and creation of prompt template

from langchain import PromptTemplate
from langchain.chains import LLMChain

template = """Answer cleanly and precisely for the question based on the context below. If the
question cannot be answered using the context provided answer
with "I don't know".

Context: The CVT is an automatic transmission that uses two pulleys with a steel belt running between them.
To continuously vary its gear ratios, the CVT simultaneously adjusts the diameter of the "drive pulley" that transmits torque from the engine and the "driven pulley" that transfers torque to the wheels.
With stepless shifting of gear ratios, the CVT can avoid the shift-shock and deliver smooth driving, making it an exceptional transmission solution.
The variable-diameter pulleys are the heart of a CVT. Each pulley is made of two 20-degree cones facing each other.
A belt rides in the groove between the two cones. V-belts are preferred if the belt is made of rubber.
V-belts get their name from the fact that the belts bear a V-shaped cross section, which increases the frictional grip of the belt.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [ ]:
print(llm(
    prompt_template.format(
        query="How belts gets their name in CVT?"
    )
))

Belts get their name in CVT from the fact that the belts bear a V-shaped cross section, which increases the frictional grip of the belt. They are referred to as V-belts.


Let's build some cool prompt templates which can be used for several use cases.

In [ ]:
python_developer_template="""
Role : You are an advanced AI with a deep understanding of Python programming.\
You are tasked to write a clean, precise, and reusable Python code for the following requirement\
Import only needed packages and use it wherever it is needed, don't use random functions which is not defined in the code.\
The code should be error-free and without any syntax errors, with a strong emphasis on efficiency.\
Return the coding block as output and it should also follow best practices for Python coding.\

User_query= {query}
 """

python_unit_test_template="""
Role : You are an advanced AI with a deep understanding of Python programming.\
Import only needed packages and use it wherever it is needed, don't use random functions which is not defined in the code.\
The code should be error-free and without any syntax errors, with a strong emphasis on efficiency.\
Return only the coding block as output and It should also follow pytest best practices for Python coding.\

User_query= {query}
"""

In [ ]:
python_dev_prompt_template = PromptTemplate(
    input_variables=["query"],
    template=python_developer_template
)

llm_chain = LLMChain(prompt=prompt_template, llm=llm)

In [ ]:
query="Write a python program to to detect outliers in given dataframe consists of numeric variables"

llm_chain.run(query)

"Here is a simple Python program using the Z-score method to detect outliers in a dataframe consisting of numeric variables. This program assumes that you are using pandas and numpy libraries.\n\n```python\nimport pandas as pd\nimport numpy as np\n\ndef detect_outliers(df):\n    outlier_indices = []\n    for col in df.columns:\n        if df[col].dtype.kind in 'biufc':\n            Q1 = df[col].quantile(0.25)\n            Q3 = df[col].quantile(0.75)\n            IQR = Q3 - Q1\n            outlier_step = 1.5 * IQR\n            outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index\n            outlier_indices.extend(outlier_list_col)\n    outlier_indices = Counter(outlier_indices)\n    multiple_outliers = list(k for k, v in outlier_indices.items() if v > 1)\n    return multiple_outliers\n\n# Assuming df is your DataFrame\noutliers = detect_outliers(df)\n```\n\nPlease replace 'df' with your actual dataframe and make sure that pandas and numpy libraries

## **Chains**

While using a language model (LLM) on its own is good for simple tasks, tackling more complex applications often requires connecting multiple LLMs either with each other or with different parts of the system.

LangChain provides various chains frameworks for connecting these components. Explore about various [Chains](https://python.langchain.com/docs/modules/chains/) in langchain and also it provides different ways to encompass [memory](https://python.langchain.com/docs/modules/memory/) into the chain which will add additional capabilities to your AI apps.

### **Simple Chain with prompt template**

Below is the method to implement a simplechain with used to generate response based on prompt templates and user's query.

In [ ]:
python_ut_prompt_template = PromptTemplate(
    input_variables=["query"],
    template=python_unit_test_template
)

llm_chain = LLMChain(prompt=prompt_template, llm=llm)

In [ ]:
query="Write a python unit test program for a function which detects outliers from the user given dataframe consists of numeric variables"

llm_chain.run(query)

"Sure, here is a simple Python unit test program for a function that detects outliers from a user-provided dataframe that consists of numeric variables. We will use the Z-score method for outlier detection. Please replace `dataframe` with your actual DataFrame name.\n\n```python\nimport pandas as pd\nimport numpy as np\nimport unittest\nfrom scipy import stats\n\ndef detect_outliers(df):\n    numeric_cols = df.select_dtypes(include=[np.number]).columns\n    outliers = {}\n    for col in numeric_cols:\n        z_scores = np.abs(stats.zscore(df[col]))\n        outliers[col] = np.where(z_scores > 3)[0]\n    return outliers\n\nclass TestOutlierDetection(unittest.TestCase):\n\n    def setUp(self):\n        self.df = pd.DataFrame({\n            'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 100],\n            'B': [10, 11, 12, 13, 14, 15, 16, 17, 18, 300],\n            'C': [20, 21, 22, 23, 24, 25, 26, 27, 28, 500]\n        })\n\n    def test_detect_outliers(self):\n        result = detect_outliers(self.d

### **Sequential Chain**


Chains are the important features of langchain which allows you to chain and package various stages of your AI responses with models and build a pipeline which allows you to generate desirable outcomes.

In [ ]:
from langchain.chains import SequentialChain
from langchain import PromptTemplate
from langchain.schema import StrOutputParser

Our idea here is to build a Sequential chain which generates a python code at stage 1 according to user query and pass it to next stage which fine tunes it to make it compliant with python typing annotations and further we are extending it to generate a unit test code for our use case mentioned in stage 1.

**Python developer template**

In [ ]:
#promt template 1 : with python_developer_template
python_developer_template=PromptTemplate.from_template("""
Role : You are an advanced AI with a deep understanding of Python programming.\
You are tasked to write a clean, precise, and reusable Python code for the following requirement\
Import only needed packages and use it wherever it is needed, don't use random functions which is not defined in the code.\
The code should be error-free and without any syntax errors, with a strong emphasis on efficiency.\
Return the coding block as output and it should also follow best practices for Python coding.\

Input : {query}
 """
)

first_chain = LLMChain(llm=llm, prompt=python_developer_template,
                     output_key="python_code"
                    )



**Python typing and lint check**

In [ ]:
#prompt template 2 : checking code with type annotation

python_type_lint_check_template= PromptTemplate.from_template( """
You are an AI model with a deep understanding of Python programming, Python type annotations, and linting standards. Your task is to review the following Python code:
Please identify any issues related to type annotations and linting in the given code.\
Then, generate a revised version of the code that is clean, precise, and free of any lint issues.\
 The revised code should also follow best practices for Python type annotations and linting.\
Remember, the goal is to enhance code readability, maintainability, and efficiency.\

Input = {python_code}

"""
)

second_chain=LLMChain(llm=llm, prompt=python_type_lint_check_template,
                     output_key="formatted_code"
                    )


**Python unit test writing template**

In [ ]:
#promt template 3 : with python_unit_testing_template

python_unit_test_template = PromptTemplate.from_template("""
Role : You are an advanced AI with a deep understanding of Python programming.\
Import only needed packages and use it wherever it is needed, don't use random functions which is not defined in the code.\
The code should be error-free and without any syntax errors, with a strong emphasis on efficiency.\
Return only the coding block as output and It should also follow pytest best practices for Python coding.\
Generate unit test for the below function:\

Input : {python_code}
"""
)
third_chain = LLMChain(llm=llm, prompt=python_unit_test_template,
                     output_key="unit_code"
                    )

Creating a sequential chain to perform actions based on stages

In [ ]:
final_chain=SequentialChain(
    chains=[first_chain,second_chain,third_chain],
    input_variables=["query"],
    output_variables=["python_code","formatted_code","unit_code"],
    verbose=False
)

In [ ]:
#sanitize function to retrieve just the python code which is executable.
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

Let's test our code pipeline,
so the flow would be from

***python_developer_template -> python_type_lint_check_template -> python_unit_test_template***

In [ ]:
r=final_chain("Write a python program to to detect outliers in given dataframe consists of numeric variables")

In [ ]:
print(_sanitize_output(r['python_code']))
print("#################################")
print(_sanitize_output(r['formatted_code']))
print("#################################")
print(_sanitize_output(r['unit_code']))




import pandas as pd
import numpy as np

def detect_outliers(df):
    # Calculate the Z-score of each value in the dataframe, this gives us a measure of how far away each value is from the mean
    z_scores = np.abs((df - df.mean()) / df.std())

    # Define a threshold to identify an outlier
    threshold = 3

    # Identify outliers
    outliers = df[(z_scores > threshold).any(axis=1)]
    
    return outliers

# Test the function
df = pd.DataFrame(np.random.randn(100, 3), columns=['A', 'B', 'C'])

outliers = detect_outliers(df)

print(outliers)

#################################

import pandas as pd
import numpy as np
from pandas import DataFrame

def detect_outliers(df: DataFrame) -> DataFrame:
    """
    Detects outliers in a given DataFrame by using the Z-Score method.

    Parameters:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: A DataFrame that only contains the outliers.
    """
    # Calculate the Z-score of each value in the dataframe,
    # this gi

###**Sequential Chains with Memory**

Implementation of SimpleMemory in Sequential chain
In this example we are going to see how we can use simple memory to store some context of information, which needs to be passed at a certain stage of our chain, eliminating the need to pass it on to every stage in prompt, which in turns affect our token size.

In [ ]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory


vision_prompt_instruction=PromptTemplate.from_template("""You are a veteran content writer and advisor who's skillfull at drafting catchy vision statement and product roadmap for new products and features.\
Be a super cool assistant and help suggesting a vision and roadmap for below context which brainstorms the market needs,\
researches similar products and giving a catchy, consice and compelling vision statement.\

Topic :{context}

"""
)

vision_chain=LLMChain(llm=llm, prompt=vision_prompt_instruction, output_key="visionmap")


translate_prompt_instruction =PromptTemplate.from_template("""You are expert in spanish and content writing, now your role is to translate the given visionmap into spanish.\
Make sure the contents are translated appropriately and sounds formal in spanish, with introducing some local terms familiar to spanish people as well.\

translate :{visionmap} """)

translate_chain=LLMChain(llm=llm, prompt=translate_prompt_instruction, output_key="translated_post")


marketer_instruct= PromptTemplate.from_template("""You are a marketing superstar, who's skillfull at promoting products with top-notch blogs and posts\
Be a super cool assistant and help creating blog post for below context which brainstorms the market need and introduces product in uber cool way\
Include a call to action and promote the blog with social media links {twitterlink}{discordlink}

Blog context :{visionmap}""")

marketer_chain=LLMChain(llm=llm, prompt=marketer_instruct, output_key="blog_post")



Here we are storing the links in the memory, which is used at the final level of chain and it is passed directly there, without need to pass it as context in every stage. Hence using a simple memory in our AI apps would impact on the tokens.


In [ ]:
chained=SequentialChain(
    memory=SimpleMemory(
        memories=({"twitterlink": "https://twitter.com/my_company","discordlink":"https://discord.com/channels/my_channel"})
    ),
    chains=[vision_chain, translate_chain, marketer_chain],
    input_variables=["context"],
    # Here we return multiple variables
    output_variables=["visionmap","translated_post","blog_post"],
    verbose=True,
)

In [ ]:
chained("We are going to launch of new product smartwatch called JARVIS which is AI assisted and does all the inference locally built-in and can replace your smartphone. ")



> Entering new SequentialChain chain...

> Finished chain.


{'context': 'We are going to launch of new product smartwatch called JARVIS which is AI assisted and does all the inference locally built-in and can replace your smartphone. ',
 'twitterlink': 'https://twitter.com/my_company',
 'discordlink': 'https://discord.com/channels/my_channel',
 'visionmap': 'Vision Statement:\n\n"Revolutionizing personal technology by integrating AI into our daily lives with JARVIS, the smartwatch that thinks locally and acts globally. We envision a world where the power of a smartphone is accessible at your wrist, providing seamless and secure access to your digital world, anytime, anywhere."\n\nProduct Roadmap:\n\n1. **Market Research & Analysis (Q1)**: Comprehensive study of the current market, understanding the needs of potential users, and analyzing the strengths and weaknesses of competing products. \n\n2. **Product Design & Development (Q2-Q3)**: Designing the smartwatch with an emphasis on user-friendly interface, advanced AI capabilities, local inferen

### **LoadSummarize chain with Clarifai vectorstore**

***What is LoadSummarize chain ?***

Assume that you are searching related to a context in your RAG chatbot which has certain information documents stored as chunks in the clarifai vectorstore.
To inspect the search results based on the retrieval, we can use the Loadsummarize chain which summarize the contents of all the retrieved chunks and gives you a good info about the particular context you are searching for.


In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document



Ingesting a documents into clarifai vectorstore DB.

In [ ]:
loader = TextLoader("../Data/history2.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

Initializing clarifai Vector DB

In [ ]:
clarifai_vector_db = Clarifai.from_documents(
    user_id="user_id",
    app_id= "app_id",
    documents=docs,
    pat="YOUR_CLARIFAI_PAT",
    number_of_docs=3,
)

Search for similar docs with the keywords or texts you want.

In [ ]:
retrieved_docs= clarifai_vector_db.similarity_search("civilization")


Using the map_reduce chain type we are summarising the output chunks.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain(retrieved_docs)

{'input_documents': [Document(page_content='A WAR FOR INTERNATIONAL FREEDOM\n\n25', metadata={'source': '/content/history2.txt'}),
  Document(page_content='A WAR FOR INTERNATIONAL FREEDOM\n\n25', metadata={'source': '/content/history2.txt'}),
  Document(page_content="Abundant materials out of our fields and our mines and our factories with which not\nonly to clothe and equip our own forces on land and sea but also to clothe and support\nour people for whom the gallant fellows under arms can no longer work, to help clothe\nand equip the armies with which we are co-operating in Europe, and to keep the looms\nand manufactories there in raw material;\nCoal to keep the fires going in ships at sea and in the furnaces of hundreds of\nfactories across the sea;\nSteel out of which to make arms and ammunition both here and there;\nRails for worn-out railways back of the fighting fronts;\nLocomotives and rolling stock to take the place of those every day going to pieces;\nEverything with which th

Summarized the contents from all the retrieved chunks for the given search context with from clarifai vectorstore.

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---